In [1]:
# Запусить Jupyter сервер на удаленной машине можно так:
# ! jupyter-notebook --ip=0.0.0.0 --no-browser .
# Последний аргумент указывает директорию, которая будет доступна через веб-интерфейс.
# Хотя магия Jupyter позволяет запустить в консоли содержимое строки
# начинающейся с восклицательного знака, нормальная работа команд из данного ноутбука возможно
# только в отдельном терминале.
# Так как вам нужно будет продолжать работу при запущенном сервере,
# нужно либо использовать несколько терминалов, либо запускать процессы в фоновом режиме,
# добавив & после команды, либо использовать tmux. 

# MPI через ipyparallel

In [2]:
# Jupyter поддерживает работу с кластером через пакет ipyparallel
# https://ipyparallel.readthedocs.io/en/latest/
# Его можно установить через PIP
# ! pip3 install ipyparallel
# После установки в интерфейсе Jupyter должна появиться вкладка IPython Clusters.
# Если этого не произошло, то нужно сделать:
# ipcluster nbextension enable

In [3]:
# ipyparallel кластер можно запусить из Jupyter из вкладки IPython Clusters.
# Альтернативно можно создать кластер из консоли:
# ! ipcluster start --profile=mpi -n 16
# По умолчанию процессы для счета создаются на локальной машине.
# Здесь мы попросили создать 16 процессов и указали, что будем использовать MPI (см. ниже).

In [4]:
# Для работы с MPI потребуется какая-либо реализация интерфейса 
# ! sudo apt install openmpi-bin
# и вспомогательная библиотека
# ! pip3 install mpi4py

In [4]:
# Чтобы получить доступ к узлам кластера, нам потребуется импортировать библиотеку
import ipyparallel as ipp

In [6]:
# Теперь мы можем создать интерфейс для работы с этими процессами.
rc = ipp.Client(profile='mpi')
# Смотрим, какие процессы были созданы:
print(f"{rc.ids}")
# Создадим "вид", для просмотра данных процессов
view = rc[:]
print(view)
# Следующая строка нужна для использования магии Jupyter
view.activate()
# Теперь мы можем выполнить содержимое ячейки на всех с помощью заклинания %%px. 

[0, 1]
<DirectView [0, 1]>


# Используем MPI

In [7]:
%%px
from mpi4py import MPI
import numpy as np

def psum(a):
    locsum = np.sum(a)
    rcvBuf = np.array(0.0,'d')
    MPI.COMM_WORLD.Allreduce([locsum, MPI.DOUBLE],
        [rcvBuf, MPI.DOUBLE],
        op=MPI.SUM)
    return rcvBuf

<AsyncResult: execute>

In [8]:
%pxresult

In [9]:
# Запускаем содержимое файла (идентично предыдущей ячейки) на каждой узле.
# view.run('psum.py')

In [10]:
# Импорт numpy нужен, так как ячейкой выше мы сделали это на удаленных машинах.
import numpy as np
# Рассылаем массив на все узлы кластера равными порциями.
view.scatter('a',np.arange(63,dtype='float'))
# Выводим содержимое массива `a` на каждой узле.
view['a']

[array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
        26., 27., 28., 29., 30., 31.]),
 array([32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42., 43., 44.,
        45., 46., 47., 48., 49., 50., 51., 52., 53., 54., 55., 56., 57.,
        58., 59., 60., 61., 62.])]

In [11]:
# Вызываем написанную нами функцию суммирования:
%px totalsum = psum(a)

<AsyncResult: execute>

In [12]:
# Смотрим результат
%pxresult

In [13]:
# Аналогично заклинанию выше.
# view.execute('totalsum = psum(a)')

In [14]:
# Выводим результат, получившийся на каждой машине:
view['totalsum']

[array(496.), array(1457.)]

# MPI на Google Colab

In [2]:
! pip install mpi4py
! pip3 install ipyparallel

     |████████████████████████████████| 1.4MB 33.4MB/s 
  Created wheel for mpi4py: filename=mpi4py-3.0.3-cp37-cp37m-linux_x86_64.whl size=2065204 sha256=eb738dfa8f194fff3e09da102caf2af7325b5f9e24f0ad2195d862acd880015f
  Stored in directory: /root/.cache/pip/wheels/18/e0/86/2b713dd512199096012ceca61429e12b960888de59818871d6
Successfully built mpi4py
     |████████████████████████████████| 204kB 32.9MB/s 


In [3]:
! ipcluster start --profile=mpi -n 2 --daemonize

2021-06-01 05:55:55.900 [IPClusterStart] Created profile dir: '/root/.ipython/profile_mpi'


Далее действуем как указанно выше для запуска MPI через Jupyter.

# CUDA через Numba

Если вы запускаете ноутбук через Google Colab, то для доступа к GPU вам нужно изменить настройки по-умолчанию: Меню > Среда выполнения > Сменить среду выполнения > Аппаратный ускоритель > GPU.

In [1]:
# Для работы с NVidia GPU проще всего использовать numba.cuda.
# Устанавливается она как обычная numba
! pip3 install numba
# но для доступа к CUDA должно быть установлено соответствующее окружение, например для Ubuntu
# ! sudo apt install nvidia-cuda-toolkit
# Проверить корректность установки можно командой
! numba -s | grep CUDA
# Документация доступна здесь:
# https://numba.pydata.org/numba-doc/latest/cuda/index.html
# Развернутые сведения об устройстве можно получить командой
# ! clinfo

__CUDA Information__
Found 1 CUDA devices
CUDA driver version                           : 11020
CUDA libraries:


In [3]:
# Импортируем необходимую библиотеку.
import numba.cuda as cuda
# Проверяем доступность numba.cuda.
print(f"{cuda.is_available()}")
# Перечисляем доступные устройства.
cuda.detect() 

True
Found 1 CUDA devices
id 0             b'Tesla T4'                              [SUPPORTED]
                      compute capability: 7.5
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True

In [5]:
# Названия функций для математический операций нужно импортировать.
import math
import numpy as np
# Пишем простую функцию для работы на GPU.
@cuda.jit
def cudasqrt(x, y):
    i = cuda.grid(1) # Оси в CUDA нумеруются с 1-ой.
    if i>=cuda.gridsize(1): return
    y[i] = math.sqrt(x[i])
# Считаем корни
x = np.arange(10, dtype=np.float32)**2
y = np.empty_like(x)
cudasqrt[1, 100](x, y) # Обязательно указываем [число блоков, число потоков на блок].
print(y)

[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
